In [ ]:
# abrindo arquivo e estabelencendo conexão com a base de dados
import json
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(user='heitor', password='123', database='yelp')
cursor = mariadb_connection.cursor()

fuser=open("user.json")
fbusiness=open("business.json")
fcheckin=open("checkin.json")
freview=open("review.json")
ftip=open("tip.json")
fphoto=open("photo.json")
user_column = ["user_id",
"name",
"review_count",
"yelping_since",
#"friends",
"useful",
"funny",
"cool",
"fans",
#"elite",
"average_stars",
"compliment_hot",
"compliment_more",
"compliment_profile",
"compliment_cute",
"compliment_list",
"compliment_note",
"compliment_plain",
"compliment_cool",
"compliment_funny",
"compliment_writer",
"compliment_photos"]

## string to int to solve the problem of redundant relantionship in friend table, 
## string_to_int(user_first_id) < string_to_int(user_second_id)
str = "Ha3iJu77CxlrFm-vQRs_8g"
def string_to_int(str):
    valor = 7
    tam = len(str)
    for i in range(tam):
        valor = 31 * valor + ord(str[i]);
    return valor;
string_to_int(str)

In [ ]:
# a variavel i serve para enviar os comandos efetivamente para o sgbd em blocos, assim sendo mais rapido os processos
i=0

for line in fuser:
    user_json=json.loads(line)
    i=i+1
    
    try:
        cursor.execute("""INSERT INTO user (
        user_id,
        name,
        review_count,
        yelping_since,
        useful,
        funny,
        cool,
        fans,
        average_stars,
        compliment_hot,
        compliment_more,
        compliment_profile,
        compliment_cute,
        compliment_list,
        compliment_note,
        compliment_plain,
        compliment_cool,
        compliment_funny,
        compliment_writer,
        compliment_photos) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                    (
        user_json["user_id"],
        user_json["name"],
        user_json["review_count"],
        user_json["yelping_since"],
        user_json["useful"],
        user_json["funny"],
        user_json["cool"],
        user_json["fans"],
        user_json["average_stars"],
        user_json["compliment_hot"],
        user_json["compliment_more"],
        user_json["compliment_profile"],
        user_json["compliment_cute"],
        user_json["compliment_list"],
        user_json["compliment_note"],
        user_json["compliment_plain"],
        user_json["compliment_cool"],
        user_json["compliment_funny"],
        user_json["compliment_writer"],
        user_json["compliment_photos"]
                    )
                    )
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        pass
        #print("Error: {}".format(error))
mariadb_connection.commit()

In [ ]:
## insert elite years of users
i=0
for line in fuser:
    i=i+1
    user_json=json.loads(line)
    try:
        for year in user_json["elite"]:
            cursor.execute("INSERT INTO elite (user_id,year) VALUES (%s,%s);",(user_json["user_id"],year))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        pass
mariadb_connection.commit()

In [ ]:
## INSERT FRIENDS
i=0
for line in fuser:
    i=i+1
    user_json=json.loads(line)
    try:
        for friend_id in user_json["friends"]:
            if string_to_int(user_json["user_id"]) < string_to_int(friend_id):
                cursor.execute("INSERT INTO friend (user_id1,user_id2) VALUES (%s,%s);",(user_json["user_id"],friend_id))
            else:
                cursor.execute("INSERT INTO friend (user_id1,user_id2) VALUES (%s,%s);",(friend_id,user_json["user_id"]))
#        if i % 100==0:
        mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
## insert business
i=0
for line in fbusiness:
    i=i+1
    business_json=json.loads(line)
    try:
        
        cursor.execute("""INSERT INTO business (
        business_id,
        name,
        address,
        city,
        state,
        `postal code`,
        latitude,
        longitude,
        stars,
        review_count,
        is_open
        ) 
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                       (
                           business_json["business_id"],
                           business_json["name"],
                           business_json["address"],
                           business_json["city"],
                           business_json["state"],
                           business_json["postal_code"],
                           business_json["latitude"],
                           business_json["longitude"],
                           business_json["stars"],
                           business_json["review_count"],
                           business_json["is_open"]
                       ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
# insert categories

i=0
for line in fbusiness:
    i=i+1
    business_json=json.loads(line)
    try:
        if business_json["categories"] == None:
            print("business_id: "+business_json["business_id"]+" -> sem categoria")
            continue
        for category_name in business_json["categories"]:
            cursor.execute("INSERT INTO category (business_id,category_name) VALUES (%s,%s);",
                           (
                               business_json["business_id"],
                               category_name
                           ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
# insert checkin

i=0
for line in fcheckin:
    i=i+1
    checkin_json=json.loads(line)
    try:
        cursor.execute("INSERT INTO checkin (business_id,date) VALUES (%s,%s);",
                       (
                           checkin_json["business_id"],
                           checkin_json["date"]
                       ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
# insert review

i=0
for line in freview:
    i=i+1
    review_json=json.loads(line)
    try:
        cursor.execute("""INSERT INTO review (
        review_id,
        user_id,
        business_id,
        stars,
        date,
        text,
        useful,
        funny,
        cool
        ) 
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
                       (
                        review_json["review_id"],
                        review_json["user_id"],
                        review_json["business_id"],
                        review_json["stars"],
                        review_json["date"],
                        review_json["text"],
                        review_json["useful"],
                        review_json["funny"],
                        review_json["cool"]
                       ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
# insert tip

i=0
for line in ftip:
    i=i+1
    tip_json=json.loads(line)
    try:
        cursor.execute("""INSERT INTO tip (
        text,
        date,
        compliment_count,
        business_id,
        user_id
        ) 
        VALUES (%s,%s,%s,%s,%s);""",
                       (
                           tip_json["text"],
                           tip_json["date"],
                           tip_json["compliment_count"],
                           tip_json["business_id"],
                           tip_json["user_id"]
                       ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()

In [ ]:
# insert photo

i=0
for line in fphoto:
    i=i+1
    photo_json=json.loads(line)
    try:
        cursor.execute("""INSERT INTO photo (
        photo_id,
        business_id,
        caption,
        label
        ) 
        VALUES (%s,%s,%s,%s);""",
                       (
                           photo_json["photo_id"],
                           photo_json["business_id"],
                           photo_json["caption"],
                           photo_json["label"]
                       ))
        if i % 10000==0:
            mariadb_connection.commit()
    except mariadb.Error as error:
        print("Error: {}".format(error))
        pass
mariadb_connection.commit()